In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [2]:
df_movie=pd.read_csv('ratings.csv')

In [3]:
df_movie

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [4]:
Utility_Maxtrix=df_movie.pivot_table(index='movieId',columns='userId',values='rating')

In [5]:
Utility_Maxtrix

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
Utility_Maxtrix=np.array(Utility_Maxtrix)

In [7]:
def Normalized_Maxtrix(arr):
    arr_temp=np.column_stack(arr)
    Avg_columns = np.zeros(arr_temp.shape[0])
    for i in range(arr_temp.shape[0]):
        Avg_columns[i] = round(np.nanmean(arr_temp[i]),2)
    for i in range(len(arr)):
        for j in range(len(arr[0])):
            arr[i][j]= arr[i][j] - Avg_columns[j]
            if np.isnan(arr[i][j]):
                arr[i][j]=0
    A_sparse=sparse.csr_matrix(arr)
    return A_sparse

In [ ]:
example_rating = np.array([[2,np.nan,np.nan,5,1,np.nan]  ### tesst
                           ,[np.nan,np.nan,3,4,3,1]
                           ,[3,1,3,np.nan,5,np.nan],
                           [np.nan,4,np.nan,np.nan,5,3]
                           ,[np.nan,1,2,3,np.nan,np.nan]
                           ,[1,np.nan,2,np.nan,2,np.nan]])

Avg_col=Normalized_Maxtrix(example_rating)
Avg_col.toarray()

array([[ 0. ,  0. ,  0. ,  1. , -2.2,  0. ],
       [ 0. ,  0. ,  0.5,  0. , -0.2, -1. ],
       [ 1. , -1. ,  0.5,  0. ,  1.8,  0. ],
       [ 0. ,  2. ,  0. ,  0. ,  1.8,  1. ],
       [ 0. , -1. , -0.5, -1. ,  0. ,  0. ],
       [-1. ,  0. , -0.5,  0. , -1.2,  0. ]])

In [8]:
Avg_col=Normalized_Maxtrix(Utility_Maxtrix)

In [9]:
Avg_col

<9724x610 sparse matrix of type '<class 'numpy.float64'>'
	with 100395 stored elements in Compressed Sparse Row format>

Numba version

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from numba import jit

# Chuyển ma trận thưa sang ma trận thông thường
def sparse_to_dense(sparse_matrix):
    return sparse_matrix.toarray()

# Hàm tính cosine similarity sử dụng Numba @jit
@jit(nopython=True, parallel = True)
def cosine_similarity_numba(matrix):
    n_cols = matrix.shape[1]
    result = np.zeros((n_cols, n_cols), dtype=np.float64)

    for r in range(n_cols):
        for c in range(r, n_cols):
            num = 0.0
            denom_a = 0.0
            denom_b = 0.0

            for k in range(matrix.shape[0]):
                num += matrix[k, r] * matrix[k, c]
                denom_a += matrix[k, r] ** 2
                denom_b += matrix[k, c] ** 2

            denom_a = np.sqrt(denom_a)
            denom_b = np.sqrt(denom_b)

            if denom_a != 0 and denom_b != 0:
                result[r, c] = num / (denom_a * denom_b)
            else:
                result[r, c] = 0.0

            # Đảm bảo tính đối xứng
            result[c, r] = result[r, c]

    return result

# Hàm chính để tính cosine similarity từ ma trận thưa đầu vào
def cosine_similarity_sparse_matrix_numba(sparse_matrix):
    dense_matrix = sparse_to_dense(sparse_matrix)
    return cosine_similarity_numba(dense_matrix)

In [ ]:
def sparse_to_dense(sparse_matrix):
    return sparse_matrix.toarray()

def cosine_similarity_numba(matrix):
    n_cols = matrix.shape[1]
    result = np.zeros((n_cols, n_cols), dtype=np.float64)

    for x in range(n_cols):
        for y in range(x, n_cols):
            num = 0.0
            denom_a = 0.0
            denom_b = 0.0

            for k in range(matrix.shape[0]):
                num += matrix[k, r] * matrix[k, c]
                denom_a += matrix[k, r] ** 2
                denom_b += matrix[k, c] ** 2

            denom_a = np.sqrt(denom_a)
            denom_b = np.sqrt(denom_b)

            if denom_a != 0 and denom_b != 0:
                result[r, c] = num / (denom_a * denom_b)
            else:
                result[r, c] = 0.0

            # Ensure symmetry
            result[c, r] = result[r, c]

    return result

# Hàm chính để tính cosine similarity từ ma trận thưa đầu vào
def cosine_similarity_sparse_matrix_numba(sparse_matrix):
    dense_matrix = sparse_to_dense(sparse_matrix)
    return cosine_similarity_numba(dense_matrix)

In [ ]:
cosine_matrix_numba = cosine_similarity_sparse_matrix_numba(Avg_col)

In [ ]:
cosine_matrix_numba

array([[ 1.00000000e+00,  1.21536489e-03,  4.76036262e-04, ...,
         7.50788575e-02, -2.59042425e-02,  1.04655379e-02],
       [ 1.21536489e-03,  1.00000000e+00,  0.00000000e+00, ...,
        -6.10340215e-03, -6.02223954e-02,  2.48713966e-02],
       [ 4.76036262e-04,  0.00000000e+00,  1.00000000e+00, ...,
        -1.31019032e-02,  0.00000000e+00,  1.95202147e-02],
       ...,
       [ 7.50788575e-02, -6.10340215e-03, -1.31019032e-02, ...,
         1.00000000e+00,  5.07450537e-02,  5.43574867e-02],
       [-2.59042425e-02, -6.02223954e-02,  0.00000000e+00, ...,
         5.07450537e-02,  1.00000000e+00, -1.24724770e-02],
       [ 1.04655379e-02,  2.48713966e-02,  1.95202147e-02, ...,
         5.43574867e-02, -1.24724770e-02,  1.00000000e+00]])

In [ ]:
np.mean(np.abs(cosine_matrix_numba - similarity_matrix.toarray()))

9.69597867359206e-18

CUDA version

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from numba import cuda, float64
import math

# Function to convert sparse matrix to dense matrix
def sparse_to_dense(sparse_matrix):
    return sparse_matrix.toarray()

# CUDA kernel to calculate cosine similarity between columns of a dense matrix
@cuda.jit
def cosine_similarity_kernel(matrix, result):
    r, c = cuda.grid(2)
    if r < result.shape[0] and c < result.shape[1]:
        num = 0.0
        denom_a = 0.0
        denom_b = 0.0

        for k in range(matrix.shape[0]):
            num += matrix[k, r] * matrix[k, c]
            denom_a += matrix[k, r] ** 2
            denom_b += matrix[k, c] ** 2

        denom_a = math.sqrt(denom_a)
        denom_b = math.sqrt(denom_b)

        if denom_a != 0 and denom_b != 0:
            result[r, c] = num / (denom_a * denom_b)
        else:
            result[r, c] = 0.0


# Function to calculate cosine similarity using CUDA
def cosine_similarity_cuda(sparse_matrix):
    dense_matrix = sparse_to_dense(sparse_matrix)
    n_cols = dense_matrix.shape[1]
    result = np.zeros((n_cols, n_cols), dtype=np.float64)

    # Configure the blocks and grids
    threadsperblock = (32, 32)
    blockspergrid_x = (result.shape[0] + threadsperblock[0] - 1) // threadsperblock[0]
    blockspergrid_y = (result.shape[0] + threadsperblock[1] - 1) // threadsperblock[1]
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    # Allocate memory on the device
    d_matrix = cuda.to_device(dense_matrix)
    d_result = cuda.device_array_like(result)

    # Launch the kernel
    cosine_similarity_kernel[blockspergrid, threadsperblock](d_matrix, d_result)

    # Copy the result back to host
    d_result.copy_to_host(result)

    return result

In [ ]:
cosine_similarity = cosine_similarity_cuda(Avg_col)

In [ ]:
cosine_similarity

array([[ 1.00000000e+00,  1.21536489e-03,  4.76036262e-04, ...,
         7.50788575e-02, -2.59042425e-02,  1.04655379e-02],
       [ 1.21536489e-03,  1.00000000e+00,  0.00000000e+00, ...,
        -6.10340215e-03, -6.02223954e-02,  2.48713966e-02],
       [ 4.76036262e-04,  0.00000000e+00,  1.00000000e+00, ...,
        -1.31019032e-02,  0.00000000e+00,  1.95202147e-02],
       ...,
       [ 7.50788575e-02, -6.10340215e-03, -1.31019032e-02, ...,
         1.00000000e+00,  5.07450537e-02,  5.43574867e-02],
       [-2.59042425e-02, -6.02223954e-02,  0.00000000e+00, ...,
         5.07450537e-02,  1.00000000e+00, -1.24724770e-02],
       [ 1.04655379e-02,  2.48713966e-02,  1.95202147e-02, ...,
         5.43574867e-02, -1.24724770e-02,  1.00000000e+00]])

Triển khai cosine similarity bằng thư viện

In [17]:
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

def cosine_similarity_sparse_matrix(sparse_matrix):
    # Transpose the matrix to compute cosine similarity between columns
    sparse_matrix_transposed = sparse_matrix.transpose()

    # Calculate cosine similarity (this will be a dense matrix)
    cosine_sim_matrix_dense = cosine_similarity(sparse_matrix_transposed)

    # Convert the dense cosine similarity matrix back to sparse format
    cosine_sim_matrix_sparse = csr_matrix(cosine_sim_matrix_dense)

    return cosine_sim_matrix_sparse

In [18]:
similarity_matrix = cosine_similarity_sparse_matrix(Avg_col)

In [ ]:
similarity_matrix.toarray()

array([[ 1.00000000e+00,  1.21536489e-03,  4.76036262e-04, ...,
         7.50788575e-02, -2.59042425e-02,  1.04655379e-02],
       [ 1.21536489e-03,  1.00000000e+00,  0.00000000e+00, ...,
        -6.10340215e-03, -6.02223954e-02,  2.48713966e-02],
       [ 4.76036262e-04,  0.00000000e+00,  1.00000000e+00, ...,
        -1.31019032e-02,  0.00000000e+00,  1.95202147e-02],
       ...,
       [ 7.50788575e-02, -6.10340215e-03, -1.31019032e-02, ...,
         1.00000000e+00,  5.07450537e-02,  5.43574867e-02],
       [-2.59042425e-02, -6.02223954e-02,  0.00000000e+00, ...,
         5.07450537e-02,  1.00000000e+00, -1.24724770e-02],
       [ 1.04655379e-02,  2.48713966e-02,  1.95202147e-02, ...,
         5.43574867e-02, -1.24724770e-02,  1.00000000e+00]])